In [1]:
# TO GAIN IMAGES AS THE RESULT OF THE AUTOENCODER
import glob
import re
import os

import metrics
import callbacks as cb

import numpy as np
from PIL import Image
import tensorflow as tf

2023-07-15 10:36:10.436011: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-15 10:36:10.504688: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-15 10:36:11.319234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def np_to_img(nparray, path, namefile):
    img = Image.fromarray(nparray)
    img_name = os.path.join(path, namefile)
    img.save(img_name)
    return img

def img_to_np(filename):
    im = Image.open(filename)
    arr = np.asarray(im)
    return arr

def angle_correction(angle):
    angle = 90 - abs(angle%180 - 90)
    return angle

In [12]:
## Data Preprocessing
train_images = glob.glob("resized_dataset/train/images/*.bmp")
test_images = glob.glob("resized_dataset/test/images/*.bmp")

x_train = []
x_test = []
y_train = []
y_test = []

for i, img in enumerate(train_images):
    arr = img_to_np(img)
    x_train.append(arr)
    data = re.search("images\/([A-Z1-9]*)_(\d.\d*)_(\d*)", img)
    _, _, angle = data.group(1), float(data.group(2)), int(data.group(3))
    angle = angle_correction(angle)
    y_train.append(angle)

for i, img in enumerate(test_images):
    arr = img_to_np(img)
    x_test.append(arr)
    data = re.search("images\/([A-Z1-9]*)_(\d.\d*)_(\d*)", img)
    _, _, angle = data.group(1), float(data.group(2)), int(data.group(3))
    angle = angle_correction(angle)
    y_test.append(angle)

x_train = np.expand_dims(np.array(x_train), axis=-1)
x_test = np.expand_dims(np.array(x_test), axis=-1)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [9]:
## Create Model
latent_dim = 512 

class Autoencoder(tf.keras.models.Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(latent_dim//4, 3, activation='relu', input_shape=[64, 64, 1]),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Conv2D(latent_dim//2, 3, activation='relu'),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Conv2D(latent_dim, 3, activation='relu'),
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(latent_dim, 3, activation='relu'), # decoder
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(latent_dim//2, 3, activation='relu'), # decoder
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(latent_dim//4, 3, activation='relu'), # decoder
        ])
        self.fullyconnected = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1),
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        output = self.fullyconnected(decoded)
        return output

autoencoder = Autoencoder(latent_dim)

In [13]:
print(x_train.shape)
print(x_test.shape)

# x_train = np.expand_dims(x_train, axis=-1)
# x_test = np.expand_dims(x_test, axis=-1)

(30412, 64, 64, 1)
(3802, 64, 64, 1)


In [14]:
## COMPILE AND TRAIN
rmse_metrics = tf.keras.metrics.RootMeanSquaredError()
autoencoder.compile(optimizer="adam",
                    loss="mse",
                    metrics=['mae', rmse_metrics, metrics.R_squared])

autoencoder.fit(x_train, y_train,
                validation_data=(x_test, y_test),
                batch_size=16, epochs=500,
                callbacks=[cb.early_stopping])

autoencoder.save("visual.h5")

Epoch 1/500


2023-07-15 10:42:54.414121: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-15 10:43:02.733723: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-15 10:43:03.161823: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-15 10:43:03.164577: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x353a1240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-15 10:43:03.164616: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-07-15 10:43:03.228583: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-15 10:43:0

1901/1901 [==============================] - 69s 27ms/step - loss: 18317.2480 - mae: 21.3241 - root_mean_squared_error: 135.3412 - R_squared: -19.8979 - val_loss: 383.1459 - val_mae: 15.9477 - val_root_mean_squared_error: 19.5741 - val_R_squared: 0.4019
Epoch 2/500
1901/1901 [==============================] - 54s 28ms/step - loss: 377.2228 - mae: 15.4712 - root_mean_squared_error: 19.4222 - R_squared: 0.3839 - val_loss: 319.2353 - val_mae: 14.0482 - val_root_mean_squared_error: 17.8672 - val_R_squared: 0.5011
Epoch 3/500
1901/1901 [==============================] - 53s 28ms/step - loss: 357.7742 - mae: 14.8907 - root_mean_squared_error: 18.9149 - R_squared: 0.4234 - val_loss: 517.5819 - val_mae: 18.3739 - val_root_mean_squared_error: 22.7504 - val_R_squared: 0.1600
Epoch 4/500
1901/1901 [==============================] - 66s 35ms/step - loss: 334.6405 - mae: 14.2247 - root_mean_squared_error: 18.2932 - R_squared: 0.4571 - val_loss: 286.3740 - val_mae: 12.7106 - val_root_mean_squared_er

KeyboardInterrupt: 

In [ ]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
import matplotlib.pyplot as plt
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
## LOAD MODEL
autoencoder = tf.keras.models.load_model("visual.h5", compile=False)
rmse_metrics = tf.keras.metrics.RootMeanSquaredError()
autoencoder.compile(optimizer="adam",
                    loss="mse",
                    metrics=['mae', rmse_metrics, metrics.R_squared])

encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()